In [ ]:
import cv2
from insightface.app import FaceAnalysis # Or specific models
import numpy as np
from IPython.display import display, Image, clear_output
from mongo_db import *

db = get_mongo_db()
(
    refname,
    refembeddings,
    classname,
    classembeddings,
    imgname,
    imgpath,
    imgbbox,
    imgcache,
    imgembeddings,
) = fetch_data_optimized(db)

# --- 1. Load Known Embeddings (Assume you've done this) ---
# Load your known_embeddings.npy and known_names.pkl
known_embeddings = classembeddings
known_names = classname

# --- 2. Initialize InsightFace ---
app = FaceAnalysis(providers=['CoreMLExecutionProvider', 'CPUExecutionProvider']) # Or specify models
app.prepare(ctx_id=0) #, det_size=(640, 640)) # Adjust as needed

# --- 3. Video Processing ---
video_path = 'your_video.mp4'
cap = cv2.VideoCapture(video_path)

display_handle = display(None, display_id=True)
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # --- 4. Face Detection and Recognition ---
        faces = app.get(frame) # Get face info

        for face in faces:
            # Get embedding for the current face
            current_embedding = face.normed_embedding
            current_embedding = np.expand_dims(current_embedding, axis=0) # Reshape for comparison

            # Compare with known embeddings (example using cosine similarity)
            similarities = np.dot(known_embeddings, current_embedding.T)
            max_similarity_idx = np.argmax(similarities)
            max_similarity = similarities[max_similarity_idx]

            name = "Unknown"
            if max_similarity > 0.5: # Set your threshold
                name = known_names[max_similarity_idx]

            # Draw bounding box and name
            bbox = face.bbox.astype(int)
            cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
            cv2.putText(frame, name, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

            # --- 5. Display Frame in Jupyter ---
            _, frame_jpeg = cv2.imencode('.jpeg', frame)
            display_handle.update(Image(data=frame_jpeg.tobytes()))
            # clear_output(wait=True) # Might be needed depending on desired update behavior

except KeyboardInterrupt:
    pass
finally:
    cap.release()
    # cv2.destroyAllWindows() # If using cv2.imshow()
